In [1]:
import pandas as pd
import numpy as np
import random

## Kai's Code

In [0]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
class dude:
      def __init__(self, occupation, age, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = {'john': 'near', 'james':'far'}
        self.network = [groupa]
        
class group:
    def __init__(self):
        self.transmission_rate = 0.0
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.memebers = []
        
        

Eddie = dude('developer', age=21)
Kai = dude('student', age=21)
Sihan = dude(occupation='student', age=22, state=0)



## Eddie's Code

In [2]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
jobs = ['student', 'doctor', 'lawyer', 'police person', 'shop assistant', 'unemployed', 'retired']
job_type = ['student', 'high-income', 'low-income', 'retired']
class dude:
    def __init__(self, occupation, age, name, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.name = name
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = network
    
    def __str__(self):
        return "Job: {}, Age: {}, State: {}".format(self.job, self.age, self.state)
        
    def __repr__(self):
        return "Job: {}, Age: {}, State: {}".format(self.job, self.age, self.state)
        
        
class group:
    def __init__(self, size, rate, code, type_):
        self.code = code
        self.type = type_
        self.transmission_rate = rate
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.size = size
        self.members = []
        
    def __str__(self):
        return "Size: {}, Infected: {}".format(self.size, self.infected_count)
    
    def __repr__(self):
        return "Code: {}, Size: {}, Infected: {}, T-rate: {:.2f}".format(self.code, self.size, self.infected_count, self.transmission_rate)
        
    def display(self):
        print("Code: {}, Size: {}, Infected: {}".format(self.code, self.size, self.infected_count))
        return

def get_group(g_list, code):
    for grp in g_list:
        if grp.code == code:
            return grp
        
def build_df(lst, what):
    if what == 'g':
        dct = {'code': [], 'type': [], 't-rate': [], 'size': []}
    
        for thing in lst:
            for tin in thing:
                dct['code'].append(tin.code)
                dct['type'].append(tin.type)
                dct['t-rate'].append(tin.transmission_rate)
                dct['size'].append(tin.size)
                
    else:
        dct = {'name': [], 'age': [], 'state': [], 'job': []}
    
        for thing in lst:
            for tin in thing:
                dct['name'].append(tin.name)
                dct['age'].append(tin.age)
                dct['state'].append(tin.state)
                dct['job'].append(tin.job)
    
    return pd.DataFrame(dct)

def create_houses(comm_size):
    houses = []
    
    for i in range(0, comm_size):
        rate = random.random()
        size = random.randint(1, 7)
        grp = group(size, rate, i, 'inner')
        houses.append(grp)
    return houses

def init_peeps(houses):
    peeps = []
    
    for i in range(0, len(houses)):
        job = random.choice(job_type)
        age = random.randint(18, 80)
        person = dude(job, age, i, network=[houses[i]])
        houses[i].members.append(person)
        peeps.append(person)
        
    return peeps

def populate_houses(people, hs):
    pep_cnt = len(people)
    
    for house in hs:
        job = house.members[0].job

        for new_member in range(0, house.size -1):
            age = abs(house.members[0].age + random.randint(-5, 5))
            person = dude(job, age, pep_cnt, network=[house])
            house.members.append(person)
            people.append(person)
            pep_cnt +=1
            
    return

def genesis(comm_size):
    houses = create_houses(comm_size)
    
    peeps = init_peeps(houses)
    
    populate_houses(peeps, houses)
    
    return houses, peeps
        

### Initial population creation and first circle

In [3]:
genesis(1000)

([Code: 0, Size: 3, Infected: 0, T-rate: 0.97,
  Code: 1, Size: 1, Infected: 0, T-rate: 0.96,
  Code: 2, Size: 4, Infected: 0, T-rate: 0.46,
  Code: 3, Size: 1, Infected: 0, T-rate: 0.34,
  Code: 4, Size: 6, Infected: 0, T-rate: 0.15,
  Code: 5, Size: 1, Infected: 0, T-rate: 0.77,
  Code: 6, Size: 7, Infected: 0, T-rate: 0.72,
  Code: 7, Size: 4, Infected: 0, T-rate: 0.78,
  Code: 8, Size: 6, Infected: 0, T-rate: 0.85,
  Code: 9, Size: 4, Infected: 0, T-rate: 0.79,
  Code: 10, Size: 4, Infected: 0, T-rate: 0.51,
  Code: 11, Size: 4, Infected: 0, T-rate: 0.99,
  Code: 12, Size: 1, Infected: 0, T-rate: 0.82,
  Code: 13, Size: 5, Infected: 0, T-rate: 0.48,
  Code: 14, Size: 7, Infected: 0, T-rate: 0.13,
  Code: 15, Size: 7, Infected: 0, T-rate: 0.86,
  Code: 16, Size: 4, Infected: 0, T-rate: 0.43,
  Code: 17, Size: 5, Infected: 0, T-rate: 0.52,
  Code: 18, Size: 6, Infected: 0, T-rate: 0.94,
  Code: 19, Size: 5, Infected: 0, T-rate: 0.18,
  Code: 20, Size: 4, Infected: 0, T-rate: 0.55,
  

### Second Circle

In [47]:
# Size of outer circles, dependent on populaiton size
ocircle_size = int(len(peeps) * 0.008 + random.randint(-5, 5))

In [48]:
# Number of outer circles, dependent on inner circle count
ocircle_count = int((len(peeps) / ocircle_size) * 0.95)

In [49]:
ocircle_size * ocircle_count

3729

In [50]:
ocircs = []
for i in range(0, ocircle_count):
    # Lower rates for outr circles
    rate = random.uniform(0,0.5)
    size = int(len(peeps) * 0.008 + random.randint(-5, 5))
    grp = group(size, rate, i, 'outer')
    ocircs.append(grp)

In [51]:
for pepe in peeps:
    nets = random.sample(ocircs, random.randint(1,3))
    for net in nets:
        net.members.append(pepe)
        pepe.network.append(net)

### Explore the Networks

In [52]:
pepsam = random.sample(peeps, 5)

for pep in pepsam:
    print(pep.name, pep)
    for net in pep.network:
        print(net.code, net)

2029 Job: high-income, Age: 33, State: 0
348 Size: 5, Infected: 0
91 Size: 27, Infected: 0
13 Size: 34, Infected: 0
2427 Job: low-income, Age: 21, State: 0
494 Size: 5, Infected: 0
29 Size: 36, Infected: 0
62 Size: 27, Infected: 0
1144 Job: retired, Age: 34, State: 0
51 Size: 4, Infected: 0
52 Size: 29, Infected: 0
23 Size: 33, Infected: 0
106 Size: 33, Infected: 0
2951 Job: student, Age: 26, State: 0
676 Size: 5, Infected: 0
96 Size: 35, Infected: 0
89 Size: 28, Infected: 0
21 Size: 27, Infected: 0
1602 Job: student, Age: 26, State: 0
200 Size: 4, Infected: 0
97 Size: 32, Infected: 0


In [53]:
get_group(ocircs, 30)

Code: 30, Size: 35, Infected: 0, T-rate: 0.32

In [14]:
for cir in ocircs:
    if cir.code == 30:
        cir.display()

Code: 30, Size: 34, Infected: 0


In [178]:
print(random.choice(random.choice(ocircs).members))

Job: high-income, Age: 60, State: 0


In [180]:
print(peeps[5].network[0])

Size: 2, Infected: 0


### Community Code Infrastructure

In [65]:
build_df([houses, ocircs], 'g')

,code,type,t-rate,size
0,0,inner,0.915067,2
1,1,inner,0.491825,4
2,2,inner,0.373567,3
3,3,inner,0.339668,5
4,4,inner,0.876795,7
...,...,...,...,...
1108,108,outer,0.148847,36
1109,109,outer,0.351030,27
1110,110,outer,0.412324,34
1111,111,outer,0.181957,27


In [68]:
build_df([peeps], 'p')

,name,age,state,job
0,0,62,0,high-income
1,1,33,0,low-income
2,2,75,0,retired
3,3,34,0,high-income
4,4,27,0,high-income
...,...,...,...,...
3921,3921,29,0,low-income
3922,3922,27,0,low-income
3923,3923,74,0,low-income
3924,3924,78,0,low-income
